In [2]:
library(downscaleR)
library(loadeR)
library(transformeR)
library(geoprocessoR)
library(raster)
library(leaflet)
library(htmltools)
library(furrr)
library(tidyverse)
library(sp)
library(leaflegend)
library(png)
library(oceanis)
library(glue)
library(grid)
library(gridExtra)
library(climate4R.indices)
library(rnaturalearth)
library(rnaturalearthdata)

In [218]:
# function to load and transform data. Ask for hurs and wind

load_data <- function (country, domain, variable, season) {
  if (!domain %in% c("AFR-22", "EAS-22"))
    stop("the specified domain is not available. Please use either AFR-22 or EAS-22")
  
  plan(multisession, workers = 18)
  
  country_shp = getData("GADM", country = country, level = 1)
  xlim <-
    c(round(country_shp@bbox[1, 1] - 2.5),
      round(country_shp@bbox[1, 2] + 2.5))  # longitude boundaries for the region of interest,
  ylim <-
    c(round(country_shp@bbox[2, 1] - 2.5),
      round(country_shp@bbox[2, 2] + 2.5))
  
  
  start <-
    paste0(
      "/home/jovyan/shared/data/CORDEX/ncml/interp/from_ESGF//CORDEX//output//",
      domain,
      "//"
    )
  run <- c("GERICS/", "ICTP/")
  GCMs <-
    c("MOHC-HadGEM2-ES/", "MPI-M-MPI-ESM-LR/", "NCC-NorESM1-M/")
  time_frame <- c("historical/", "rcp26/", "rcp85/")
  file_end <-
    c("GERICS-REMO2015//v1//day//", "ICTP-RegCM4-7//v0/day//")
  
  # simple functions to return models with the same time-frane
  common_dates <- function(data) {
    dates.all = c()
    
    for (imodel in 1:length(data)) {
      dates.all = c(dates.all, substr(data[[imodel]]$Dates$start, 1, 10))
    }
    
    dates.common = substr(data[[1]]$Dates$start, 1, 10)
    for (imodel in 2:length(data)) {
      aux = intersect(substr(data[[imodel - 1]]$Dates$start, 1, 10),
                      substr(data[[imodel]]$Dates$start, 1, 10))
      dates.common = intersect(dates.common, aux)
    }
    for (imodel in 1:length(data)) {
      ind = which(!is.na(match(
        substr(data[[imodel]]$Dates$start, 1, 10), dates.common
      )))
      data[[imodel]] = subsetDimension(data[[imodel]], dimension = "time", indices = ind)
    }
    return(bindGrid(data, dimension = "member"))
    
  }
  
  # building the dataset
  
  message(
    paste(
      Sys.time(),
      "Loading the required 18 GCM-RCM climate models, namely MOHC-HadGEM2-ES, MPI-M-MPI-ESM-LR, NCC-NorESM1-M
downscaled with RCMs REMO2015 and RegCM4-4 (3 GCMs*2 RCMs*3 scenarios(2 RCPs and historical data)).
The process is currently parallelized using 18 cores but it can still requires up to 3 minutes..."
    )
  )
  
  files <- paste0(start, run) %>%
    map(., ~ paste0(.x, GCMs) %>%
          map(., ~ paste0(.x, time_frame) %>%
                map(., ~ paste0(.x, file_end)))) %>%
    unlist() %>%
    str_replace(
      .,
      "MPI-M-MPI-ESM-LR/",
      ifelse(
        str_detect(., "ICTP"),
        "MPI-M-MPI-ESM-MR/",
        "MPI-M-MPI-ESM-LR/"
      )
    ) %>%
    list.files(., full.names = TRUE)
  
  df1 <-
    tibble(
      path = unlist(files),
      RCP = str_extract(path, "rcp\\d+"),
      simulation = str_extract(path, "GERICS")
    ) %>%
    mutate(
      RCP = ifelse(is.na(RCP), "historical", RCP),
      simulation = ifelse(is.na(simulation), "ICTP", simulation)
    ) %>%
    mutate(
      climate_data = future_map(path,   function(x)  {
        if (str_detect(x, "historical"))
          suppressMessages(
            loadGridData(
              dataset = x,
              var = variable,
              years = 1980:2005,
              lonLim = xlim,
              latLim = ylim,
              season = season
            )
          )
        else
          suppressMessages(
            loadGridData(
              dataset = x,
              var = variable,
              years = 2006:2099,
              lonLim = xlim,
              latLim = ylim,
              season = season
            )
          )
      }),
      climate_data = future_map(climate_data, function(x) {
        if (str_detect(variable, "pr"))
          gridArithmetics(x, 86400, operator = "*")
        else
          gridArithmetics(x, 273.15, operator = "-")
        
      })
    )
  
  message(paste(Sys.time(), "Done"))
  
  message(
    paste(
      Sys.time(),
      "Performing ensemble mean and standard deviation. Can take up to 7 minutes"
    )
  )
  
  plan(multisession, workers = 3)
  
  df2 <- df1 %>%
    group_by(RCP) %>%
    summarise(
      models = list(climate_data),
      models_mbrs = map(models, ~ common_dates(.x)),
      models_mean = future_map(models_mbrs, ~ suppressMessages(aggregateGrid(
        .x, aggr.mem = list(FUN = "mean", na.rm = TRUE)
      ))),
      models_sd = future_map(models_mbrs, ~ suppressMessages(aggregateGrid(
        .x, aggr.mem = list(FUN = "sd", na.rm = TRUE)
      )))
    )  %>%
    select(-models) %>%
    mutate(obs = list(suppressMessages(
      loadGridData(
        "/home/jovyan/shared/data/observations/W5E5/v1.0/w5e5_v1.0.ncml",
        var = variable,
        years = 1980:2005,
        lonLim = xlim,
        latLim = ylim,
        season = season
      )
    )))
  
  message(paste(Sys.time(), "Done"))
  
  return(df2)
  
}

In [ ]:
data_out <- load_data(country="Cape Verde", domain="AFR-22", variable="pr", season=1:12)

2021-12-28 16:15:50 Loading the required 18 GCM-RCM climate models, namely MOHC-HadGEM2-ES, MPI-M-MPI-ESM-LR, NCC-NorESM1-M 
downscaled with RCMs REMO2015 and RegCM4-4 (3 GCMs*2 RCMs*3 scenarios(2 RCPs and historical data)). 
The process is currently parallelized using 18 cores but it can still requires up to 3 minutes...

2021-12-28 16:20:58 Done

2021-12-28 16:20:58 Performing ensemble mean and standard deviation. Can take up to 7 minutes

